In [3]:
import matplotlib.cm as cm
import numpy as np
import corner
import matplotlib.pyplot as plt
import emcee
import math
from astropy.visualization import astropy_mpl_style
plt.style.use(astropy_mpl_style)
import scipy.optimize as op
import matplotlib as mpl
from astropy.table import Table
import matplotlib.gridspec as gridspec
from astropy.io import fits
from numpy import full, nan
from traceback_functions import lnlikeV, lnpriorV, lnprobV

In [7]:
hdul = fits.open('all_OBA.fits')
hdul = hdul[1].data
t = Table(hdul)

In [9]:
ra = t['ra']
dec = t['dec']
l = t['l']
b = t['b']
Plx = t['plx']
ePlx = t['e_plx']
pmRA = t['pmra_1']
pmDE = t['pmde_1']
epmRA = t['e_pmra_1']
epmDE = t['e_pmde_1']

In [11]:
### Define central RA, Dec, l, b, proper motions, cluster distance, cluster RV

Lc= 265.08
Bc= 1.4
Dcluster = 1000
RVc50 = -20.75
RVc84, RVc16 = 20.12, -0.418

In [13]:
T = np.array([-0.0548755604162154,-0.8734370902348850,-0.4838350155487132,0.4941094278755837,-0.4448296299600112,0.7469822444972189,-0.8676661490190047,-0.1980763734312015,0.4559837761750669]).reshape(3,3)
k = 4.74057
Tinv = np.linalg.inv(T)

In [15]:
N = len(ra)  
traceback_indices = [] 
vLcorr=[]
vBcorr=[]
vLcorr_p=[]
vBcorr_p=[]
vLcorr_n=[]
vBcorr_n=[]
PMRA_=[]
PMDEC_=[]
EPMRA_=[]
EPMDEC_=[]
PARALLAXES_=[]
PML=[]
PMB=[]

for idx in range(0,len(ra)):
    
    #################### Here you may choose only to perform the transformation on sources with reliable astrometry with an 'if' statement  #############
    
    # if ruwe[idx] < 1.4 and prob[idx]>0.89 and CGcluster[idx] == name and np.isnan(Gmag[idx])==False:
        Ra = np.deg2rad(ra[idx])
        DEC = np.deg2rad(dec[idx])
        L = np.deg2rad(l[idx])
        B = np.deg2rad(b[idx])
        PLX = Plx[idx]
        EPLX = ePlx[idx]
        PMRA = pmRA[idx]
        PMDEC = pmDE[idx]
        EPMRA = epmRA[idx]
        EPMDEC = epmDE[idx]
        PMRA_.append(PMRA)
        PMDEC_.append(PMDEC)
        EPMRA_.append(EPMRA)
        EPMDEC_.append(EPMDEC)
        PARALLAXES_.append(PLX)
        
        Picrs=np.array([float(-np.sin(Ra)),float(np.cos(Ra)),0])
        Qicrs=np.array([float(-np.cos(Ra)*np.sin(DEC)),float(-np.sin(Ra)*np.sin(DEC)),float(np.cos(DEC))])
        Pgal=np.array([float(-np.sin(L)),float(np.cos(L)),0])
        Qgal=np.array([float(-np.cos(L)*np.sin(B)),float(-np.sin(L)*np.sin(B)),float(np.cos(B))])
        icrs=PMRA*np.cos(DEC)*Picrs+PMDEC*Qicrs
        GAL = np.dot(T,icrs)
        pmL=np.dot(Pgal.transpose(),GAL)/np.cos(B)
        pmB=np.dot(Qgal.transpose(),GAL)
        PML.append(pmL)
        PMB.append(pmB)
        
        vL=(k*Dcluster*pmL)/1000
        vB=(k*Dcluster*pmB)/1000
        
        #print(idx)
        m_true2 = [vL,vB]
        zp = np.arange(-100., 100., 1.)
        
        ndim, nwalkers = 3, 50
        pos1 = [np.array([np.random.choice(zp)+1000./PLX,np.random.choice(zp)+m_true2[0],np.random.choice(zp)+m_true2[1]]) for i in range(nwalkers)]
        sampler1 = emcee.EnsembleSampler(nwalkers, ndim, lnprobV, args=(PLX, EPLX, PMRA, PMDEC, EPMRA, EPMDEC, DEC,  Pgal, Qgal, Picrs, Qicrs))
        sampler1.run_mcmc(pos1, 500, rstate0=np.random.get_state())   
        burnin = 200
        samples1 = sampler1.chain[:, burnin:, :].reshape((-1, ndim))
        
        m_mcmc4 = list(map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                                     zip(*np.percentile(samples1, [16, 50, 84],
                                                        axis=0))))[1]
        
        m_mcmc5 = list(map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                                     zip(*np.percentile(samples1, [16, 50, 84],
                                                        axis=0))))[2]
            
        #print(m_mcmc4)
        #print(m_mcmc5)
        
        vLcorr.append(m_mcmc4[0]-RVc50*np.cos(np.deg2rad(Bc))*np.sin(np.deg2rad(Lc-l[idx])))
        vBcorr.append(m_mcmc5[0]-RVc50*(np.sin(np.deg2rad(Bc))*np.cos(np.deg2rad(b[idx]))-np.sin(np.deg2rad(b[idx]))*np.cos(np.deg2rad(Bc))*np.cos(np.deg2rad(Lc-l[idx]))))
        vLcorr_p.append(np.sqrt(m_mcmc4[1]**2+((RVc84-RVc50)*np.cos(np.deg2rad(Bc))*np.sin(np.deg2rad(Lc-l[idx])))**2))
        vBcorr_p.append(np.sqrt(m_mcmc5[1]**2+((RVc84-RVc50)*(np.sin(np.deg2rad(Bc))*np.cos(np.deg2rad(b[idx]))-np.sin(np.deg2rad(b[idx]))*np.cos(np.deg2rad(Bc))*np.cos(np.deg2rad(Lc-l[idx]))))**2))
        vLcorr_n.append(np.sqrt(m_mcmc4[2]**2+((RVc50-RVc16)*np.cos(np.deg2rad(Bc))*np.sin(np.deg2rad(Lc-l[idx])))**2))
        vBcorr_n.append(np.sqrt(m_mcmc5[2]**2+((RVc50-RVc16)*(np.sin(np.deg2rad(Bc))*np.cos(np.deg2rad(b[idx]))-np.sin(np.deg2rad(b[idx]))*np.cos(np.deg2rad(Bc))*np.cos(np.deg2rad(Lc-l[idx]))))**2))


vLcorr=np.asarray(vLcorr)
vBcorr=np.asarray(vBcorr)
vLcorr_p=np.asarray(vLcorr_p)
vBcorr_p=np.asarray(vBcorr_p)
vLcorr_n=np.asarray(vLcorr_n)
vBcorr_n=np.asarray(vBcorr_n)

In [16]:
### You can estimate the cluster central tangential velocities by using the median, as well as estimate the uncertainty on the median values themselves

medianvL=[]
medianvB=[]
n=0
while n<10000:
    a=np.random.normal(0, 1, size=(1,len(vLcorr))) 
    pos = [i*j for i, j in zip(a.clip(0), vLcorr_p)]
    neg = [i*j for i, j in zip(a.clip(max=0), vLcorr_n)]
    vLcorre = [i + j for i, j in zip(pos, neg)]
    medianvL.append(np.median(vLcorr+vLcorre))
    a=np.random.normal(0, 1, size=(1,len(vBcorr))) 
    pos = [i*j for i, j in zip(a.clip(0), vBcorr_p)]
    neg = [i*j for i, j in zip(a.clip(max=0), vBcorr_n)]
    vBcorre = [i + j for i, j in zip(pos, neg)]
    medianvB.append(np.median(vBcorr+vBcorre))
    n+=1
    
vLc16, vLc50, vLc84 = np.percentile(np.sort(medianvL), [16, 50, 84])
vBc16, vBc50, vBc84 = np.percentile(np.sort(medianvB), [16, 50, 84])

In [17]:
#### This bit separates the velocities into expansion and rotation components, so we can calculate the average expansion and rotation velocities


vLx =[]
vBx =[]
vLr =[]
vBr =[]
vLxp =[]
vBxp =[]
vLrp =[]
vBrp =[]
vLxn =[]
vBxn =[]
vLrn =[]
vBrn =[]
vLxe =[]
vBxe =[]
vLre =[]
vBre =[]
RadD4=[]
RadD4_p=[]
RadD4_n=[]
RadD4_e=[]
RadV4=[]
RadV4_p=[]
RadV4_n=[]
RadV4_e=[]
RotV4=[]
RotV4_p=[]
RotV4_n=[]
RotV4_e=[]

RadVun4=[]
RadVun4_p=[]
RadVun4_n=[]
RadVun4_e=[]



for idx in range(0,len(l)):
    Langle = np.arctan2(b[idx]-Bc,(l[idx]-Lc)*(np.cos(np.deg2rad(Bc))))
    vLex=[]
    vBex=[]
    vLro=[]
    vBro=[]
    RadDist=[]
    RadVel=[]
    RadVelun=[]
    RotVel=[]
    X5=[]
    Y5=[]
    U5=[]
    V5=[]
    n=0
    while n<1000:
        a=np.random.normal(0, 1, size=(1,1)) 
        pos = a.clip(0)*(vLcorr_p[idx])
        neg = a.clip(max=0)*(vLcorr_n[idx])
        vLcorre = pos+neg
        vLcorridx = vLcorr[idx]+vLcorre
        a=np.random.normal(0, 1, size=(1,1)) 
        pos = a.clip(0)*(vBcorr_p[idx])
        neg = a.clip(max=0)*(vBcorr_n[idx])
        vBcorre = pos+neg
        vBcorridx = vBcorr[idx]+vBcorre
        a=np.random.normal(0, 1, size=(1,1)) 
        pos = a.clip(0)*(vLc84-vLc50)
        neg = a.clip(max=0)*(vLc50-vLc16)
        vLe = pos+neg
        vLc = vLc50+vLe
        a=np.random.normal(0, 1, size=(1,1)) 
        pos = a.clip(0)*(vBc84-vBc50)
        neg = a.clip(max=0)*(vBc50-vBc16)
        vBe = pos+neg
        vBc = vBc50+vBe
        
        #### Option to also include uncertainty on the cluster distance #####
        
        # a=np.random.normal(0, 1, size=(1,1)) 
        # pos = a.clip(0)*(Dcluster_bs84-Dcluster_bs50)
        # neg = a.clip(max=0)*(Dcluster_bs50-Dcluster_bs16)
        # Dclustere = pos+neg
        # Dclusterc = Dcluster_bs50+Dclustere
        n+=1
        
        
        vLangle=np.arctan2(vBcorridx-vBc,(vLcorridx-vLc)*(np.cos(np.deg2rad(Bc))))
        Xangle=Langle-vLangle
        vLex.append(np.sqrt(((vLcorridx-vLc)*(np.cos(np.deg2rad(Bc))))**2+(vBcorridx-vBc)**2)*np.cos(Xangle)*np.cos(Langle)+vLc)
        vBex.append(np.sqrt(((vLcorridx-vLc)*(np.cos(np.deg2rad(Bc))))**2+(vBcorridx-vBc)**2)*np.cos(Xangle)*np.sin(Langle)+vBc)
        vLro.append(np.sqrt(((vLcorridx-vLc)*(np.cos(np.deg2rad(Bc))))**2+(vBcorridx-vBc)**2)*np.sin(Xangle)*np.sin(Langle)+vLc)
        vBro.append(np.sqrt(((vLcorridx-vLc)*(np.cos(np.deg2rad(Bc))))**2+(vBcorridx-vBc)**2)*np.sin(Xangle)*-np.cos(Langle)+vBc)  
        
        
        r=np.sqrt(((l[idx]-Lc)*(np.cos(np.deg2rad(Bc))))**2+(b[idx]-Bc)**2)
        rho = np.sqrt(((vLcorridx-vLc)*(np.cos(np.deg2rad(Bc))))**2+(vBcorridx-vBc)**2)*np.cos(Xangle)#*np.cos(Dangle) #Should this be multiplied by *np.cos(Dangle) for the 1d expansion analysis?
        RadDist.append(np.tan(np.deg2rad(r))* Dcluster) #Dclusterc)  #Converted to pcs using the cluster distance
        RadVel.append(rho)
        RotVel.append(np.sqrt(((vLcorridx-vLc)*(np.cos(np.deg2rad(Bc))))**2+(vBcorridx-vBc)**2)*np.sin(Xangle))#*np.cos(Dangle))
        
    
        

    vL16, vL50, vL84 = np.percentile(np.sort(vLex), [16, 50, 84])
    vB16, vB50, vB84 = np.percentile(np.sort(vBex), [16, 50, 84])
    vLx.append(vL50)
    vLxp.append(vL84-vL50)
    vLxn.append(vL50-vL16)
    vLxe.append((vL84-vL16)/2)
    vBx.append(vB50)
    vBxp.append(vB84-vB50)
    vBxn.append(vB50-vB16)
    vBxe.append((vB84-vB16)/2)
    
    
    vL16, vL50, vL84 = np.percentile(np.sort(vLro), [16, 50, 84])
    vB16, vB50, vB84 = np.percentile(np.sort(vBro), [16, 50, 84])
    vLr.append(vL50)
    vLrp.append(vL84-vL50)
    vLrn.append(vL50-vL16)
    vLre.append((vL84-vL16)/2)
    vBr.append(vB50)
    vBrp.append(vB84-vB50)
    vBrn.append(vB50-vB16)
    vBre.append((vB84-vB16)/2)
    
    RD16, RD50, RD84 = np.percentile(RadDist, [16, 50, 84])
    RV16, RV50, RV84 = np.percentile(RadVel, [16, 50, 84])
    RotV16, RotV50, RotV84 = np.percentile(RotVel, [16, 50, 84])
    RadD4.append(RD50)
    RadD4_p.append(RD84-RD50)
    RadD4_n.append(RD50-RD16)
    RadD4_e.append((RD84-RD16)/2)
    RadV4.append(RV50)
    RadV4_p.append(RV84-RV50)
    RadV4_n.append(RV50-RV16)
    RadV4_e.append((RV84-RV16)/2)
    RadDerr4 = [RadD4_n,RadD4_p]
    RadVerr4 = [RadV4_n,RadV4_p]
    RotV4.append(RotV50)
    RotV4_p.append(RotV84-RotV50)
    RotV4_n.append(RotV50-RotV16)
    RotV4_e.append((RotV84-RotV16)/2)
    RotVerr4 = [RotV4_n,RotV4_p]
    

    
vLx =np.asarray(vLx)
vBx =np.asarray(vBx)
vLr =np.asarray(vLr)
vBr =np.asarray(vBr)
vLxp =np.asarray(vLxp)
vBxp =np.asarray(vBxp)
vLrp =np.asarray(vLrp)
vBrp =np.asarray(vBrp)
vLxn =np.asarray(vLxn)
vBxn =np.asarray(vBxn)
vLrn =np.asarray(vLrn)
vBrn =np.asarray(vBrn)
vLxe =np.asarray(vLxe)
vBxe =np.asarray(vBxe)
vLre =np.asarray(vLre)
vBre =np.asarray(vBre)

RadD4=np.asarray(RadD4)
RadD4_p=np.asarray(RadD4_p)
RadD4_n=np.asarray(RadD4_n)
RadV4=np.asarray(RadV4)
RadV4_p=np.asarray(RadV4_p)
RadV4_n=np.asarray(RadV4_n)
RotV4=np.asarray(RotV4)
RotV4_p=np.asarray(RotV4_p)
RotV4_n=np.asarray(RotV4_n)

ExpV=[]
n=0
while n<100000:
    a=np.random.normal(0, 1, size=(1,len(RadV4))) 
    pos = [i*j for i, j in zip(a.clip(0), RadV4_p)]
    neg = [i*j for i, j in zip(a.clip(max=0), RadV4_n)]
    RadV4e = [i + j for i, j in zip(pos, neg)]
    ExpV.append(np.median(RadV4+RadV4e))
    n+=1
    
ExpV16, ExpV50, ExpV84 = np.percentile(np.sort(ExpV), [16, 50, 84])

RotV=[]
n=0
while n<100000:
    a=np.random.normal(0, 1, size=(1,len(RotV4))) 
    pos = [i*j for i, j in zip(a.clip(0), RotV4_p)]
    neg = [i*j for i, j in zip(a.clip(max=0), RotV4_n)]
    RotV4e = [i + j for i, j in zip(pos, neg)]
    RotV.append(np.median(RotV4+RotV4e))
    n+=1
    
RotV16, RotV50, RotV84 = np.percentile(np.sort(RotV), [16, 50, 84])

print(f"Expansion Velocity 2D = {ExpV50:.3f}⁺{ExpV84 - ExpV50:.3f}₋{ExpV50 - ExpV16:.3f} km/s")
print(f"Rotation Velocity 2D  = {RotV50:.3f}⁺{RotV84 - RotV50:.3f}₋{RotV50 - RotV16:.3f} km/s")

Expansion Velocity 2D = -0.522⁺0.167₋0.165 km/s
Rotation Velocity 2D  = 1.162⁺0.242₋0.248 km/s


In [18]:
#### Now the code to calculate closest approach to the cluster center, traceback time, etc. ####

KinAge2D=[]
PA=[]
PA2=[]
KinAge2D_CA=[]
CA=[]
KinAge2D_p=[]
PA_p=[]
PA2_p=[]
KinAge2D_CA_p=[]
CA_p=[]
KinAge2D_n=[]
PA_n=[]
PA2_n=[]
KinAge2D_CA_n=[]
CA_n=[]
ANGLE2=[]
ANGLE2_p=[]
ANGLE2_n=[]
RadD4_2=[]
CApc=[]
CApc_p=[]
CApc_n=[]
l_=[]
b_=[]
TBDistance=[]
TBDistance_p=[]
TBDistance_n=[]
vLcorr__=[]
vBcorr__=[]
vLcorr__p=[]
vBcorr__p=[]
vLcorr__n=[]
vBcorr__n=[]

for idx in range(0,len(RadD4)):
    positionAngle=(np.rad2deg(np.arctan2((b[idx]-Bc),(l[idx]-Lc)*np.cos(np.deg2rad(Bc)))))
    PA2.append(positionAngle)
    if RadD4[idx]/RadV4[idx] >0: #### Selects only stars moving outwards for the traceback ####
        traceback_indices.append(idx)
        RadD4_2.append(RadD4[idx])
        KinAge2D__=[]
        PA__=[]
        KinAge2D_CA__=[]
        KinAge2D_CA_ct__=[]
        CA__=[]
        ANGLE2__=[]
        CApc__=[]
        TBDistance__=[]
        vLcorr__.append(vLcorr[idx])
        vBcorr__.append(vBcorr[idx])
        vLcorr__p.append(vLcorr_p[idx])
        vBcorr__p.append(vBcorr_p[idx])
        vLcorr__n.append(vLcorr_n[idx])
        vBcorr__n.append(vBcorr_n[idx])
        n=0
        while n<1000:
            a=np.random.normal(0, 1, size=(1,1)) 
            pos = a.clip(0)*(vLcorr_p[idx])
            neg = a.clip(max=0)*(vLcorr_n[idx])
            vLcorre = pos+neg
            vLcorridx = vLcorr[idx]+vLcorre
            a=np.random.normal(0, 1, size=(1,1)) 
            pos = a.clip(0)*(vBcorr_p[idx])
            neg = a.clip(max=0)*(vBcorr_n[idx])
            vBcorre = pos+neg
            vBcorridx = vBcorr[idx]+vBcorre
            a=np.random.normal(0, 1, size=(1,1)) 
            pos = a.clip(0)*(vLc84-vLc50)
            neg = a.clip(max=0)*(vLc50-vLc16)
            vLe = pos+neg
            vLc = vLc50+vLe
            a=np.random.normal(0, 1, size=(1,1)) 
            pos = a.clip(0)*(vBc84-vBc50)
            neg = a.clip(max=0)*(vBc50-vBc16)
            vBe = pos+neg
            vBc = vBc50+vBe
            
            #### Option to also include uncertainty on the cluster distance #####
            
            # a=np.random.normal(0, 1, size=(1,1)) 
            # pos = a.clip(0)*(Dcluster84-Dcluster50)
            # neg = a.clip(max=0)*(Dcluster50-Dcluster16)
            # Dclustere = pos+neg
            # Dclusterc = Dcluster50+Dclustere
            
            a=np.random.normal(0, 1, size=(1,1)) 
            pos = a.clip(0)*(RadD4_p[idx])
            neg = a.clip(max=0)*(RadD4_n[idx])
            RadD4e = pos+neg
            RadD4idx = RadD4[idx]+RadD4e
            a=np.random.normal(0, 1, size=(1,1)) 
            pos = a.clip(0)*(RadV4_p[idx])
            neg = a.clip(max=0)*(RadV4_n[idx])
            RadV4e = pos+neg
            RadV4idx = RadV4[idx]+RadV4e
            n+=1
            
            KinAge2D__.append((1/1.023)*RadD4idx/RadV4idx)
            PA__.append(positionAngle)
            a_CA=((vBcorridx-vBc)/((vLcorridx-vLc)*np.cos(np.deg2rad(Bc))))
            b_CA=-1
            c_CA=b[idx]-Bc - ((vBcorridx-vBc)/((vLcorridx-vLc)*np.cos(np.deg2rad(Bc))))*(l[idx]-Lc)*np.cos(np.deg2rad(Bc))
            d_CA=np.abs(c_CA)/np.sqrt(a_CA**2+b_CA**2)
            CA__.append(d_CA) 
            DistanceTravelled=np.sqrt((b[idx]-Bc)**2+(l[idx]-Lc)**2-d_CA**2)
            TBDistance__.append(DistanceTravelled)
            VelTravelled=np.sqrt((vBcorridx-vBc)**2+((vLcorridx-vLc)*np.cos(np.deg2rad(Bc)))**2)
            
            CApc__.append((np.tan(np.deg2rad(d_CA))*Dcluster))  # Dclusterc))
            KinAge2D_CA__.append((1/1.023)*(np.tan(np.deg2rad(DistanceTravelled))*Dcluster)/VelTravelled) # Dclusterc)/VelTravelled)
            pmAngle=(np.rad2deg(np.arctan2((vBcorridx-vBc),(vLcorridx-vLc)*np.cos(np.deg2rad(Bc)))))
            if pmAngle-positionAngle > 180:
                ANGLE2__.append(pmAngle-positionAngle-180)
            elif pmAngle-positionAngle < -180:
                ANGLE2__.append(pmAngle-positionAngle+180)
            else:
                ANGLE2__.append(pmAngle-positionAngle)
            
        
        KinAge2D16, KinAge2D50, KinAge2D84 = np.percentile(np.sort(KinAge2D__), [16, 50, 84])
        PA16, PA50, PA84 = np.percentile(np.sort(PA__), [16, 50, 84])
        CA16, CA50, CA84 = np.percentile(np.sort(CA__), [16, 50, 84])
        KinAge2D_CA16, KinAge2D_CA50, KinAge2D_CA84 = np.percentile(np.sort(KinAge2D_CA__), [16, 50, 84])
        ANGLE216, ANGLE250, ANGLE284 = np.percentile(np.sort(ANGLE2__), [16, 50, 84])
        CApc16, CApc50, CApc84 = np.percentile(np.sort(CApc__), [16, 50, 84])
        TBDistance__16, TBDistance__50, TBDistance__84 = np.percentile(np.sort(TBDistance__), [16, 50, 84])
        
        KinAge2D.append(KinAge2D50)
        KinAge2D_p.append(KinAge2D84-KinAge2D50)
        KinAge2D_n.append(KinAge2D50-KinAge2D16)
        PA.append(PA50)
        PA_p.append(PA84-PA50)
        PA_n.append(PA50-PA16)
        CA.append(CA50) 
        CA_p.append(CA84-CA50)
        CA_n.append(CA50-CA16)
        CApc.append(CApc50) 
        CApc_p.append(CApc84-CApc50)
        CApc_n.append(CApc50-CApc16)
        KinAge2D_CA.append(KinAge2D_CA50)
        KinAge2D_CA_p.append(KinAge2D_CA84-KinAge2D_CA50)
        KinAge2D_CA_n.append(KinAge2D_CA50-KinAge2D_CA16)
        ANGLE2.append(ANGLE250)
        ANGLE2_p.append(ANGLE284-ANGLE250)
        ANGLE2_n.append(ANGLE250-ANGLE216)
        l_.append(l[idx])
        b_.append(b[idx])
        TBDistance.append(TBDistance__50)
        TBDistance_p.append(TBDistance__84-TBDistance__50)
        TBDistance_n.append(TBDistance__50-TBDistance__16)

KinAge2D_all = full(N, nan)
KinAge2D_p_all = full(N, nan)
KinAge2D_n_all = full(N, nan)
PA_all = full(N, nan)
PA_p_all = full(N, nan)
PA_n_all = full(N, nan)
CA_all = full(N, nan)
CA_p_all = full(N, nan)
CA_n_all = full(N, nan)
CApc_all = full(N, nan)
CApc_p_all = full(N, nan)
CApc_n_all = full(N, nan)
KinAge2D_CA_all = full(N, nan)
KinAge2D_CA_p_all = full(N, nan)
KinAge2D_CA_n_all = full(N, nan)
ANGLE2_all = full(N, nan)
ANGLE2_p_all = full(N, nan)
ANGLE2_n_all = full(N, nan)
TBDistance_all = full(N, nan)
TBDistance_p_all = full(N, nan)
TBDistance_n_all = full(N, nan)

for i, idx in enumerate(traceback_indices):
    KinAge2D_all[idx] = KinAge2D[i]
    KinAge2D_p_all[idx] = KinAge2D_p[i]
    KinAge2D_n_all[idx] = KinAge2D_n[i]
    PA_all[idx] = PA[i]
    PA_p_all[idx] = PA_p[i]
    PA_n_all[idx] = PA_n[i]
    CA_all[idx] = CA[i]
    CA_p_all[idx] = CA_p[i]
    CA_n_all[idx] = CA_n[i]
    CApc_all[idx] = CApc[i]
    CApc_p_all[idx] = CApc_p[i]
    CApc_n_all[idx] = CApc_n[i]
    KinAge2D_CA_all[idx] = KinAge2D_CA[i]
    KinAge2D_CA_p_all[idx] = KinAge2D_CA_p[i]
    KinAge2D_CA_n_all[idx] = KinAge2D_CA_n[i]
    ANGLE2_all[idx] = ANGLE2[i]
    ANGLE2_p_all[idx] = ANGLE2_p[i]
    ANGLE2_n_all[idx] = ANGLE2_n[i]
    TBDistance_all[idx] = TBDistance[i]
    TBDistance_p_all[idx] = TBDistance_p[i]
    TBDistance_n_all[idx] = TBDistance_n[i]

In [32]:
tab = Table(
    [
        ra, dec, l, b,
        vLcorr, vLcorr_p, vLcorr_n,
        vBcorr, vBcorr_p, vBcorr_n,
        vLx, vLxp, vLxn, vLxe,
        vBx, vBxp, vBxn, vBxe,
        vLr, vLrp, vLrn, vLre,
        vBr, vBrp, vBrn, vBre,
        RadD4, RadD4_p, RadD4_n,
        RadV4, RadV4_p, RadV4_n,
        RotV4, RotV4_p, RotV4_n,
        KinAge2D_all, KinAge2D_p_all, KinAge2D_n_all,
        PA_all, PA_p_all, PA_n_all,
        CA_all, CA_p_all, CA_n_all,
        CApc_all, CApc_p_all, CApc_n_all,
        KinAge2D_CA_all, KinAge2D_CA_p_all, KinAge2D_CA_n_all,
        ANGLE2_all, ANGLE2_p_all, ANGLE2_n_all,
        TBDistance_all, TBDistance_p_all, TBDistance_n_all
    ],
    names=[
        "ra", "dec", "l", "b",
        "vLcorr", "vLcorr_p", "vLcorr_n", 
        "vBcorr", "vBcorr_p", "vBcorr_n", 
        "vLx", "vLx_p", "vLx_n", "vLx_e",
        "vBx", "vBx_p", "vBx_n", "vBx_e",
        "vLr", "vLr_p", "vLr_n", "vLr_e",
        "vBr", "vBr_p", "vBr_n", "vBr_e",
        "RadD4", "RadD4_p", "RadD4_n",
        "RadV4", "RadV4_p", "RadV4_n",
        "RotV4", "RotV4_p", "RotV4_n",
        "KinAge2D", "KinAge2D_p", "KinAge2D_n",
        "PA", "PA_p", "PA_n",
        "CA", "CA_p", "CA_n",
        "CApc", "CApc_p", "CApc_n",
        "KinAge2D_CA", "KinAge2D_CA_p", "KinAge2D_CA_n",
        "ANGLE2", "ANGLE2_p", "ANGLE2_n",
        "TBDistance", "TBDistance_p", "TBDistance_n"
    ]
)

# Optional: units and metadata -------------------------------------
tab['ra'].unit  = 'deg'
tab['dec'].unit = 'deg'
tab.meta['comment'] = "Full cluster kinematic/traceback catalogue"

# ------------------------------------------------------------------
# 4.  Save in your preferred formats -------------------------------
# ------------------------------------------------------------------
tab.write("cluster_kinematic_traceback.fits", format="fits", overwrite=True)

In [53]:
hdul = fits.open('RCW_36.fits')
hdul = hdul[1].data
t = Table(hdul)

ra = t['RA']
dec = t['DEC']
l = t['l']
b = t['b']
Plx = t['parallax']
ePlx = t['parallax_error']
pmRA = t['pmra']
pmDE = t['pmdec']
epmRA = t['pmra_error']
epmDE = t['pmdec_error']

mask = ~np.isnan(ra) & ~np.isnan(dec) & ~np.isnan(l) & ~np.isnan(b) & ~np.isnan(Plx) & ~np.isnan(ePlx) & ~np.isnan(pmRA) & ~np.isnan(pmDE) & ~np.isnan(epmRA) & ~np.isnan(epmDE)

ra = ra[mask]
dec = dec[mask]
l = l[mask]
b = b[mask]
Plx = Plx[mask]
ePlx = ePlx[mask]
pmRA = pmRA[mask]
pmDE = pmDE[mask]
epmRA = epmRA[mask]
epmDE = epmDE[mask]


vLcorr=[]
vBcorr=[]
vLcorr_p=[]
vBcorr_p=[]
vLcorr_n=[]
vBcorr_n=[]
PMRA_=[]
PMDEC_=[]
EPMRA_=[]
EPMDEC_=[]
PARALLAXES_=[]
PML=[]
PMB=[]

for idx in range(0,len(ra)):
    
    #################### Here you may choose only to perform the transformation on sources with reliable astrometry with an 'if' statement  #############
    
    # if ruwe[idx] < 1.4 and prob[idx]>0.89 and CGcluster[idx] == name and np.isnan(Gmag[idx])==False:
        Ra = np.deg2rad(ra[idx])
        DEC = np.deg2rad(dec[idx])
        L = np.deg2rad(l[idx])
        B = np.deg2rad(b[idx])
        PLX = Plx[idx]
        EPLX = ePlx[idx]
        PMRA = pmRA[idx]
        PMDEC = pmDE[idx]
        EPMRA = epmRA[idx]
        EPMDEC = epmDE[idx]
        PMRA_.append(PMRA)
        PMDEC_.append(PMDEC)
        EPMRA_.append(EPMRA)
        EPMDEC_.append(EPMDEC)
        PARALLAXES_.append(PLX)
        
        Picrs=np.array([float(-np.sin(Ra)),float(np.cos(Ra)),0])
        Qicrs=np.array([float(-np.cos(Ra)*np.sin(DEC)),float(-np.sin(Ra)*np.sin(DEC)),float(np.cos(DEC))])
        Pgal=np.array([float(-np.sin(L)),float(np.cos(L)),0])
        Qgal=np.array([float(-np.cos(L)*np.sin(B)),float(-np.sin(L)*np.sin(B)),float(np.cos(B))])
        icrs=PMRA*np.cos(DEC)*Picrs+PMDEC*Qicrs
        GAL = np.dot(T,icrs)
        pmL=np.dot(Pgal.transpose(),GAL)/np.cos(B)
        pmB=np.dot(Qgal.transpose(),GAL)
        PML.append(pmL)
        PMB.append(pmB)
        
        vL=(k*Dcluster*pmL)/1000
        vB=(k*Dcluster*pmB)/1000
        
        #print(idx)
        m_true2 = [vL,vB]
        zp = np.arange(-100., 100., 1.)
        
        ndim, nwalkers = 3, 50
        pos1 = [np.array([np.random.choice(zp)+1000./PLX,np.random.choice(zp)+m_true2[0],np.random.choice(zp)+m_true2[1]]) for i in range(nwalkers)]
        sampler1 = emcee.EnsembleSampler(nwalkers, ndim, lnprobV, args=(PLX, EPLX, PMRA, PMDEC, EPMRA, EPMDEC, DEC,  Pgal, Qgal, Picrs, Qicrs))
        sampler1.run_mcmc(pos1, 500, rstate0=np.random.get_state())   
        burnin = 200
        samples1 = sampler1.chain[:, burnin:, :].reshape((-1, ndim))
        
        m_mcmc4 = list(map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                                     zip(*np.percentile(samples1, [16, 50, 84],
                                                        axis=0))))[1]
        
        m_mcmc5 = list(map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                                     zip(*np.percentile(samples1, [16, 50, 84],
                                                        axis=0))))[2]
            
        #print(m_mcmc4)
        #print(m_mcmc5)
        
        vLcorr.append(m_mcmc4[0]-RVc50*np.cos(np.deg2rad(Bc))*np.sin(np.deg2rad(Lc-l[idx])))
        vBcorr.append(m_mcmc5[0]-RVc50*(np.sin(np.deg2rad(Bc))*np.cos(np.deg2rad(b[idx]))-np.sin(np.deg2rad(b[idx]))*np.cos(np.deg2rad(Bc))*np.cos(np.deg2rad(Lc-l[idx]))))
        vLcorr_p.append(np.sqrt(m_mcmc4[1]**2+((RVc84-RVc50)*np.cos(np.deg2rad(Bc))*np.sin(np.deg2rad(Lc-l[idx])))**2))
        vBcorr_p.append(np.sqrt(m_mcmc5[1]**2+((RVc84-RVc50)*(np.sin(np.deg2rad(Bc))*np.cos(np.deg2rad(b[idx]))-np.sin(np.deg2rad(b[idx]))*np.cos(np.deg2rad(Bc))*np.cos(np.deg2rad(Lc-l[idx]))))**2))
        vLcorr_n.append(np.sqrt(m_mcmc4[2]**2+((RVc50-RVc16)*np.cos(np.deg2rad(Bc))*np.sin(np.deg2rad(Lc-l[idx])))**2))
        vBcorr_n.append(np.sqrt(m_mcmc5[2]**2+((RVc50-RVc16)*(np.sin(np.deg2rad(Bc))*np.cos(np.deg2rad(b[idx]))-np.sin(np.deg2rad(b[idx]))*np.cos(np.deg2rad(Bc))*np.cos(np.deg2rad(Lc-l[idx]))))**2))


vLcorr=np.asarray(vLcorr)
vBcorr=np.asarray(vBcorr)
vLcorr_p=np.asarray(vLcorr_p)
vBcorr_p=np.asarray(vBcorr_p)
vLcorr_n=np.asarray(vLcorr_n)
vBcorr_n=np.asarray(vBcorr_n)

clustervL = np.median(vLcorr)
clustervB = np.median(vBcorr)

print(f' cluster average vLcorr : {clustervL:.2f} [km/s]')
print(f' cluster average vBcorr : {clustervB:.2f} [km/s]')

 cluster average vLcorr : -48.89 [km/s]
 cluster average vBcorr : -9.74 [km/s]


In [55]:
### You can estimate the cluster central tangential velocities by using the median, as well as estimate the uncertainty on the median values themselves

medianvL=[]
medianvB=[]
n=0
while n<10000:
    a=np.random.normal(0, 1, size=(1,len(vLcorr))) 
    pos = [i*j for i, j in zip(a.clip(0), vLcorr_p)]
    neg = [i*j for i, j in zip(a.clip(max=0), vLcorr_n)]
    vLcorre = [i + j for i, j in zip(pos, neg)]
    medianvL.append(np.median(vLcorr+vLcorre))
    a=np.random.normal(0, 1, size=(1,len(vBcorr))) 
    pos = [i*j for i, j in zip(a.clip(0), vBcorr_p)]
    neg = [i*j for i, j in zip(a.clip(max=0), vBcorr_n)]
    vBcorre = [i + j for i, j in zip(pos, neg)]
    medianvB.append(np.median(vBcorr+vBcorre))
    n+=1
    
vLc16, vLc50, vLc84 = np.percentile(np.sort(medianvL), [16, 50, 84])
vBc16, vBc50, vBc84 = np.percentile(np.sort(medianvB), [16, 50, 84])

In [57]:
#### This bit separates the velocities into expansion and rotation components, so we can calculate the average expansion and rotation velocities


vLx =[]
vBx =[]
vLr =[]
vBr =[]
vLxp =[]
vBxp =[]
vLrp =[]
vBrp =[]
vLxn =[]
vBxn =[]
vLrn =[]
vBrn =[]
vLxe =[]
vBxe =[]
vLre =[]
vBre =[]
RadD4=[]
RadD4_p=[]
RadD4_n=[]
RadD4_e=[]
RadV4=[]
RadV4_p=[]
RadV4_n=[]
RadV4_e=[]
RotV4=[]
RotV4_p=[]
RotV4_n=[]
RotV4_e=[]

RadVun4=[]
RadVun4_p=[]
RadVun4_n=[]
RadVun4_e=[]



for idx in range(0,len(l)):
    Langle = np.arctan2(b[idx]-Bc,(l[idx]-Lc)*(np.cos(np.deg2rad(Bc))))
    vLex=[]
    vBex=[]
    vLro=[]
    vBro=[]
    RadDist=[]
    RadVel=[]
    RadVelun=[]
    RotVel=[]
    X5=[]
    Y5=[]
    U5=[]
    V5=[]
    n=0
    while n<1000:
        a=np.random.normal(0, 1, size=(1,1)) 
        pos = a.clip(0)*(vLcorr_p[idx])
        neg = a.clip(max=0)*(vLcorr_n[idx])
        vLcorre = pos+neg
        vLcorridx = vLcorr[idx]+vLcorre
        a=np.random.normal(0, 1, size=(1,1)) 
        pos = a.clip(0)*(vBcorr_p[idx])
        neg = a.clip(max=0)*(vBcorr_n[idx])
        vBcorre = pos+neg
        vBcorridx = vBcorr[idx]+vBcorre
        a=np.random.normal(0, 1, size=(1,1)) 
        pos = a.clip(0)*(vLc84-vLc50)
        neg = a.clip(max=0)*(vLc50-vLc16)
        vLe = pos+neg
        vLc = vLc50+vLe
        a=np.random.normal(0, 1, size=(1,1)) 
        pos = a.clip(0)*(vBc84-vBc50)
        neg = a.clip(max=0)*(vBc50-vBc16)
        vBe = pos+neg
        vBc = vBc50+vBe
        
        #### Option to also include uncertainty on the cluster distance #####
        
        # a=np.random.normal(0, 1, size=(1,1)) 
        # pos = a.clip(0)*(Dcluster_bs84-Dcluster_bs50)
        # neg = a.clip(max=0)*(Dcluster_bs50-Dcluster_bs16)
        # Dclustere = pos+neg
        # Dclusterc = Dcluster_bs50+Dclustere
        n+=1
        
        
        vLangle=np.arctan2(vBcorridx-vBc,(vLcorridx-vLc)*(np.cos(np.deg2rad(Bc))))
        Xangle=Langle-vLangle
        vLex.append(np.sqrt(((vLcorridx-vLc)*(np.cos(np.deg2rad(Bc))))**2+(vBcorridx-vBc)**2)*np.cos(Xangle)*np.cos(Langle)+vLc)
        vBex.append(np.sqrt(((vLcorridx-vLc)*(np.cos(np.deg2rad(Bc))))**2+(vBcorridx-vBc)**2)*np.cos(Xangle)*np.sin(Langle)+vBc)
        vLro.append(np.sqrt(((vLcorridx-vLc)*(np.cos(np.deg2rad(Bc))))**2+(vBcorridx-vBc)**2)*np.sin(Xangle)*np.sin(Langle)+vLc)
        vBro.append(np.sqrt(((vLcorridx-vLc)*(np.cos(np.deg2rad(Bc))))**2+(vBcorridx-vBc)**2)*np.sin(Xangle)*-np.cos(Langle)+vBc)  
        
        
        r=np.sqrt(((l[idx]-Lc)*(np.cos(np.deg2rad(Bc))))**2+(b[idx]-Bc)**2)
        rho = np.sqrt(((vLcorridx-vLc)*(np.cos(np.deg2rad(Bc))))**2+(vBcorridx-vBc)**2)*np.cos(Xangle)#*np.cos(Dangle) #Should this be multiplied by *np.cos(Dangle) for the 1d expansion analysis?
        RadDist.append(np.tan(np.deg2rad(r))* Dcluster) #Dclusterc)  #Converted to pcs using the cluster distance
        RadVel.append(rho)
        RotVel.append(np.sqrt(((vLcorridx-vLc)*(np.cos(np.deg2rad(Bc))))**2+(vBcorridx-vBc)**2)*np.sin(Xangle))#*np.cos(Dangle))
        
    
        

    vL16, vL50, vL84 = np.percentile(np.sort(vLex), [16, 50, 84])
    vB16, vB50, vB84 = np.percentile(np.sort(vBex), [16, 50, 84])
    vLx.append(vL50)
    vLxp.append(vL84-vL50)
    vLxn.append(vL50-vL16)
    vLxe.append((vL84-vL16)/2)
    vBx.append(vB50)
    vBxp.append(vB84-vB50)
    vBxn.append(vB50-vB16)
    vBxe.append((vB84-vB16)/2)
    
    
    vL16, vL50, vL84 = np.percentile(np.sort(vLro), [16, 50, 84])
    vB16, vB50, vB84 = np.percentile(np.sort(vBro), [16, 50, 84])
    vLr.append(vL50)
    vLrp.append(vL84-vL50)
    vLrn.append(vL50-vL16)
    vLre.append((vL84-vL16)/2)
    vBr.append(vB50)
    vBrp.append(vB84-vB50)
    vBrn.append(vB50-vB16)
    vBre.append((vB84-vB16)/2)
    
    RD16, RD50, RD84 = np.percentile(RadDist, [16, 50, 84])
    RV16, RV50, RV84 = np.percentile(RadVel, [16, 50, 84])
    RotV16, RotV50, RotV84 = np.percentile(RotVel, [16, 50, 84])
    RadD4.append(RD50)
    RadD4_p.append(RD84-RD50)
    RadD4_n.append(RD50-RD16)
    RadD4_e.append((RD84-RD16)/2)
    RadV4.append(RV50)
    RadV4_p.append(RV84-RV50)
    RadV4_n.append(RV50-RV16)
    RadV4_e.append((RV84-RV16)/2)
    RadDerr4 = [RadD4_n,RadD4_p]
    RadVerr4 = [RadV4_n,RadV4_p]
    RotV4.append(RotV50)
    RotV4_p.append(RotV84-RotV50)
    RotV4_n.append(RotV50-RotV16)
    RotV4_e.append((RotV84-RotV16)/2)
    RotVerr4 = [RotV4_n,RotV4_p]
    

    
vLx =np.asarray(vLx)
vBx =np.asarray(vBx)
vLr =np.asarray(vLr)
vBr =np.asarray(vBr)
vLxp =np.asarray(vLxp)
vBxp =np.asarray(vBxp)
vLrp =np.asarray(vLrp)
vBrp =np.asarray(vBrp)
vLxn =np.asarray(vLxn)
vBxn =np.asarray(vBxn)
vLrn =np.asarray(vLrn)
vBrn =np.asarray(vBrn)
vLxe =np.asarray(vLxe)
vBxe =np.asarray(vBxe)
vLre =np.asarray(vLre)
vBre =np.asarray(vBre)

RadD4=np.asarray(RadD4)
RadD4_p=np.asarray(RadD4_p)
RadD4_n=np.asarray(RadD4_n)
RadV4=np.asarray(RadV4)
RadV4_p=np.asarray(RadV4_p)
RadV4_n=np.asarray(RadV4_n)
RotV4=np.asarray(RotV4)
RotV4_p=np.asarray(RotV4_p)
RotV4_n=np.asarray(RotV4_n)

ExpV=[]
n=0
while n<100000:
    a=np.random.normal(0, 1, size=(1,len(RadV4))) 
    pos = [i*j for i, j in zip(a.clip(0), RadV4_p)]
    neg = [i*j for i, j in zip(a.clip(max=0), RadV4_n)]
    RadV4e = [i + j for i, j in zip(pos, neg)]
    ExpV.append(np.median(RadV4+RadV4e))
    n+=1
    
ExpV16, ExpV50, ExpV84 = np.percentile(np.sort(ExpV), [16, 50, 84])

RotV=[]
n=0
while n<100000:
    a=np.random.normal(0, 1, size=(1,len(RotV4))) 
    pos = [i*j for i, j in zip(a.clip(0), RotV4_p)]
    neg = [i*j for i, j in zip(a.clip(max=0), RotV4_n)]
    RotV4e = [i + j for i, j in zip(pos, neg)]
    RotV.append(np.median(RotV4+RotV4e))
    n+=1
    
RotV16, RotV50, RotV84 = np.percentile(np.sort(RotV), [16, 50, 84])

print(f"Expansion Velocity 2D = {ExpV50:.3f}⁺{ExpV84 - ExpV50:.3f}₋{ExpV50 - ExpV16:.3f} km/s")
print(f"Rotation Velocity 2D  = {RotV50:.3f}⁺{RotV84 - RotV50:.3f}₋{RotV50 - RotV16:.3f} km/s")

Expansion Velocity 2D = 1.446⁺4.015₋4.409 km/s
Rotation Velocity 2D  = -6.409⁺3.124₋3.462 km/s
